# Launch object detection *training*

## Installation

You should select a runtime with GPU first: (Runtime -> Change Runtime Type = GPU)

In [ ]:
%tensorflow_version 1.x

In [0]:
!apt-get install libsm6 libxext6 libxrender-dev libcap-dev ffmpeg
!pip install --upgrade git+https://github.com/surfriderfoundationeurope/mot.git

## Training

### Downloading dataset

In [0]:
import wget
from zipfile import ZipFile

wget.download("http://files.heuritech.com/raw_files/dataset_surfrider_cleaned.zip", ".")

with ZipFile('dataset_surfrider_cleaned.zip', 'r') as zipObj:
     zipObj.extractall()

### Download pretrained weights on COCO

In [0]:
wget.download("http://models.tensorpack.com/FasterRCNN/COCO-MaskRCNN-R50FPN2x.npz", ".")

### Launch the training

See this [file](https://github.com/surfriderfoundationeurope/mot/blob/master/src/mot/object_detection/README.md) to choose the architecture you want and the according pre trained weights. The weights downloaded above correspond to ResNet50-FPN with 2X scheduling.  
Since we are not interested in segmentation, we set MODE_MASK=False. Also, because the dataset is pretty small, we don't need to train the network for a lot of steps. We decrease the learning rate at steps (250,500,750) * 8 GPUs, correspoding to the steps (2000, 4000, 6000).

In [0]:
!python3 -m mot.object_detection.train --load COCO-MaskRCNN-R50FPN2x.npz --logdir resnet50_fpn --config DATA.BASEDIR=dataset_surfrider_cleaned MODE_MASK=False TRAIN.LR_SCHEDULE=250,500,750

### Visualize predictions

In [0]:
!mkdir plastic_trained_resnet50_fpn

In [0]:
wget.download("https://files.heuritech.com/raw_files/surfrider/resnet50_fpn/model-6000.index", "plastic_trained_resnet50_fpn")
wget.download("https://files.heuritech.com/raw_files/surfrider/resnet50_fpn/model-6000.data-00000-of-00001", "plastic_trained_resnet50_fpn")

In [0]:
!python3 -m mot.object_detection.predict --load plastic_trained_resnet50_fpn/model-6000 --predict dataset_surfrider_cleaned/Images_md5/9ddc58812851ad643114930524601f10 --config DATA.BASEDIR=dataset_surfrider_cleaned MODE_MASK=False

In [0]:
from IPython.display import Image
Image('output.png')